### Final code to get the ancestors of each child GO term. This will only select those ancestors that have the highest generation level and also are present in the go_parent_data database
#### Then the output dataframe was saved as go_ancestor_data.csv

In [160]:
import pandas as pd
from goatools.obo_parser import GODag

In [161]:
#load the go children data file
go_child_df = pd.read_csv("/Users/zaidur/Documents/Sequence_Project/aeromonasBact/uniprot_go_childs.csv")
#load the go parent database
go_parent_data = pd.read_csv("/Users/zaidur/Documents/Sequence_Project/aeromonasBact/go_parent_data.csv")

In [162]:
#Reading the csv file turns the lists (go_terms column) into strings.
#We need to convert the go_terms column into list

import ast

go_child_df['go_terms'] = go_child_df['go_terms'].apply(ast.literal_eval)

In [164]:
import pandas as pd
from goatools.obo_parser import GODag

#function for getting the ancestors and their generation level as a dictionary
def get_ancestors(go_term, ancestors_dict):
    ancestors_dict[go_term] = go_dag[go_term].level
    for parent in go_dag[go_term].parents:
        get_ancestors(parent.id, ancestors_dict)

# GO Database location
obo_fname = '/Users/zaidur/Documents/Sequence_Project/aeromonasBact/go-basic.obo'

# Load the GO graph
go_dag = GODag(obo_file=obo_fname)

go_child_df['go_parents'] = '' # initialize new column

for index in range(len(go_child_df)):
    go_list = []
    for go_term in go_child_df.loc[index, 'go_terms']:
        
        ##### makes dictionary of all the ancestors of that GO term
        ancestors_dict = {}
        get_ancestors(go_term, ancestors_dict)
        
        
        ###### now only the higest level ancestors are chosen that are also present in the go_parent_data database
        filtered_ancestors_dict = {k: v for k, v in ancestors_dict.items() if k in go_parent_data['go_id'].tolist()}

        # Get the highest level
        highest_level = max(filtered_ancestors_dict.values())

        # Get all ancestors with the highest level
        highest_level_ancestors = [k for k, v in filtered_ancestors_dict.items() if v == highest_level]
        
        go_list.extend(highest_level_ancestors)
        
    
    go_child_df.at[index, 'go_parents'] = go_list
    
go_child_df   

/Users/zaidur/Documents/Sequence_Project/aeromonasBact/go-basic.obo: fmt(1.2) rel(2023-05-10) 46,490 Terms


,id,sequence,go_terms,go_parents
0,A0A068FVC1,MNIIKTAIPDVHIFEPKVFFDERGFFFESFNHKLFEEAVGYSVNFV...,"[GO:0008830, GO:0019305]","[GO:0016853, GO:1901135, GO:0055086]"
1,A0A068FZD0,MTTQSSKSRVFVAGHRGMVGSAICRQLAQRTDIELVVRSRSELDLT...,"[GO:0016853, GO:0042351, GO:0050577, GO:0070401]","[GO:0016853, GO:1901135, GO:0055086, GO:001649..."
2,A0A068FZK6,METSGLVAFVGTALAIACLRPLSAKLQLVDLPNQRKQHVGAIPLIG...,"[GO:0000287, GO:0005886, GO:0009243, GO:000927...","[GO:0003674, GO:0005886, GO:0005975, GO:000561..."
3,A0A075P9Z7,MNLTELKQKPITDLLQLAEEMGIENMARSRKQDVIFSLLKKHAKSG...,"[GO:0003723, GO:0004386, GO:0005524, GO:000582...","[GO:0003723, GO:0003824, GO:0140657, GO:000367..."
4,A0A075PBX8,MQISVNEFLTPRHIDVQVVSPTRAKITLEPLERGFGHTLGNALRRI...,"[GO:0000428, GO:0003677, GO:0003899, GO:000573...","[GO:0005575, GO:0003677, GO:0140098, GO:000557..."
...,...,...,...,...
29776,Q7BJX9,MDIYMSRYEEITQQLIFSPKTWLITGVAGFIGSNLLEKLLKLNQVV...,"[GO:0000166, GO:0003974, GO:0009243]","[GO:0003674, GO:0016853, GO:0005975]"
29777,Q8UVZ1,MEQANLYEVAPRPLMTSLVQNQQNPYIYKDTAGDLSEICENENSID...,"[GO:0000978, GO:0000981, GO:0001228, GO:000188...","[GO:0003677, GO:0140110, GO:0140110, GO:004885..."
29778,Q9L5A4,MKQTSLALAITALLSTLPSALVQANEGCAPLTGKESGMDIGRSSTE...,"[GO:0004252, GO:0005576, GO:0006508, GO:004259...","[GO:0016787, GO:0140096, GO:0005576, GO:000815..."
29779,R1GTS7,MFARLEGRPVLLVGGGEVALRKARLLLAAGARLTLVSPVLASEFDE...,"[GO:0004851, GO:0009236, GO:0019354, GO:003225...","[GO:0016740, GO:0006766, GO:0008150, GO:000815..."


In [165]:
go_child_df.to_csv("/Users/zaidur/Documents/Sequence_Project/aeromonasBact/go_ancestor_data.csv", index=False)